# Homework №2

This homework will be dedicated to **ASR & Co**.

In general, you may implement any ASR model that was discussed in the lecture,
but we recommend to implement **QuartzNet**.

## **Important aspects (model)**
1) Pay attention on different length of utterances. P.S. **masking**.
    
2) A good ASR is a robust ASR, so we ask you to implement and use at least **4 types of augmentations** (P.S. 2 seminar).

3) Also, to get better quality, we ask you to implement a **beam search** for better decoding.

4) (Bonus) As a bonus you can use **BPE** instead of Char. You can use SentencePiece, HuggingFace or YouTokenToMe.

5) (Bonus) As a bonus you can take pretrained **LM** (or train yourself) and fusing LM with ASR.
    Way of fusing you may choose yourself.

## **Important aspects (code)**
1) You already know about pytorch-lighting (I hope :)) but you are not allowed to use it in this homework.

2) Try to write code more structurally and cleanly !

3) Good logging of experiments save your nerves and time,
    so we ask you to use **W&B** and log at least loss, WER, CER and pairs (audio -- recognized text).
    **Do not remove** the logs until we have checked your work and given you a grade!

4) We also ask you to organize your code in github repo with Docker and setup.py. You can use my template https://github.com/markovka17/dl-start-pack.

5) Your work **must be** reproducable, so fix seed, save the weights of model, and etc.

6) In the end of your work write inference utils. Anyone should be able to take your weight, load it into the model and run it on some audio track.

## Data

CommonVoice Mozilla: https://commonvoice.mozilla.org/en/datasets
Он весит 50 гигабайт. В нем значительно больше коротких записей, что должно ускорить сходимость методов на нем.
Можно обучиться на нем всем или просто отщипнуть себе кусочек.

Все еще настоятельно рекомендую препроцессить датасет и выкидывать все записи длиннее N-секунд (а если вы учите не CTC, то стоит еще дополнительно выкидывать все записи длиннее  K символов), чтобы максимизировать размер батча.

##### config

https://github.com/NVIDIA/NeMo/blob/main/examples/asr/conf/quartznet_15x5.yaml

### Начинаем решение

In [1]:
import wandb

In [172]:
import string
import re
import math
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import librosa
import asrtoolkit
import torch_optimizer



from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch import distributions

from collections import Counter
from IPython import display as display_
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/kirill/.local/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random', 'test', 'f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [221]:
BATCH_SIZE = 80
NUM_EPOCHS = 10
N_MELS     = 64

In [222]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

### Датасет

In [223]:
class TrainDataset(torch.utils.data.Dataset):
    """Custom competition dataset."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.answers = pd.read_csv(csv_file, '\t')
        self.transform = transform


    def __len__(self):
        return len(self.answers)


    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        utt_name = 'cv-corpus-5.1-2020-06-22/en/clips/' + self.answers.loc[idx, 'path']
        utt = torchaudio.load(utt_name)[0].squeeze()
        
        if len(utt.shape) != 1:
            print(utt.shape)
            print(utt)
            utt = utt[1]
            
        answer = self.answers.loc[idx, 'sentence']

        if self.transform:
            utt = self.transform(utt)

        sample = {'utt': utt, 'answer': answer}
        return sample

In [224]:
class TestDataset(torch.utils.data.Dataset):
    """Custom competition dataset."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.names = pd.read_csv(csv_file, '\t')
        self.transform = transform


    def __len__(self):
        return len(self.names)


    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        utt_name = 'cv-corpus-5.1-2020-06-22/en/clips/' + self.names.loc[idx, 'path']
        utt = torchaudio.load(utt_name)[0].squeeze()
  

        if self.transform:
            utt = self.transform(utt)

        sample = {'utt': utt}
        return sample

#### Аугментации

In [225]:
def transform_tr(wav):
    aug_num = torch.randint(low=0, high=3, size=(1,)).item()
    augs = [
        lambda x: x,
        lambda x: (x + distributions.Normal(0, 0.01).sample(x.size())).clamp_(-1, 1),
        lambda x: torchaudio.transforms.Vol(.1)(x)
    ]
    
    return augs[aug_num](wav)

#### Визуализация

In [226]:
def viz(wav):
    figsize(20, 5)
    plot(wav)
    plt.show()

    display_.display(display_.Audio(wav, rate=48000, normalize=False))

#### Класс для кодировки текста (0 - blank_token)

In [227]:
class TextTransform:
    def __init__(self):
        self.char_dict = {}
        self.index_dict = {}
        
        self.char_dict['\''] = 0
        self.index_dict[0] = '\''
        self.char_dict[' '] = 1
        self.index_dict[1] = ' '
        for i, let in enumerate(string.ascii_lowercase):
            self.index_dict[i + 2] = let
            self.char_dict[let] = i + 2
            
    def text_to_int(self, text):
        labels = []
        for let in text:
            labels.append(self.char_dict[let])
        return labels
    
    def int_to_text(self, labels):
        text = []
        for num in labels:
            text.append(self.index_dict[num])
        return text

#### Collate_fn

In [243]:
#win_len=1024, hop_len=256
# counting len of MelSpec before doing it (cause of padding)
def mel_len(x):
    return int(x // 256) + 1

In [229]:
def preprocess_data(data):
    text_transform = TextTransform()
    wavs = []
    input_lens = []
    labels = []
    label_lens = []
    
    for el in data:
        wavs.append(el['utt'])
        input_lens.append(math.ceil(mel_len(el['utt'].shape[0]) / 2))    # cause of stride 2
        label = torch.Tensor(text_transform.text_to_int(el['answer']))
        labels.append(label)
        label_lens.append(len(label))

    wavs = pad_sequence(wavs, batch_first=True)
    labels = pad_sequence(labels, batch_first=True)
    
    return wavs, input_lens, labels, label_lens    

In [230]:
# Loading data and loaders
my_dataset = TrainDataset(csv_file='train_preprocessed.tsv', transform=transform_tr)
print('all train+val samples:', len(my_dataset))
test_dataset = TestDataset(csv_file='cv-corpus-5.1-2020-06-22/en/test.tsv', transform=None)

all train+val samples: 435947


100k files ~ 10GB

95_percentile_len = 412416.0

Отсортированные по длинне звука индексы объектов. 

In [236]:
# sorted indexes
with open('sorted.npy', 'rb') as f:
    s = np.load(f)

In [237]:
to_save = s[:100000][:, 0]

In [239]:
my_dataset = torch.utils.data.Subset(my_dataset, to_save)
train_set, val_set = torch.utils.data.random_split(my_dataset, [85000, 15000])

#### Даталоадеры

In [241]:
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE,
                          shuffle=True, collate_fn=preprocess_data, drop_last=True,
                          num_workers=0, pin_memory=True)

val_loader = DataLoader(val_set, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=preprocess_data, drop_last=True,
                        num_workers=0, pin_memory=True)

test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

## Модель

#### Фикс сидов и смотрим кол-во параметров

In [245]:
def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
set_seed(21)

In [281]:
def count_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    return sum([np.prod(p.size()) for p in model_parameters])

#### Вспомогательные блоки

In [247]:
def conv_bn_act(in_size, out_size, kernel_size, stride=1, dilation=1):
    return nn.Sequential(
        nn.Conv1d(in_size, out_size, kernel_size, stride, dilation=dilation),
        nn.BatchNorm1d(out_size),
        nn.ReLU()
    )


def sepconv_bn(in_size, out_size, kernel_size, stride=1, dilation=1, padding=None):
    if padding is None:
        padding = (kernel_size-1)//2
    return nn.Sequential(
        torch.nn.Conv1d(in_size, in_size, kernel_size, 
                        stride=stride, dilation=dilation, groups=in_size,
                        padding=padding),
        torch.nn.Conv1d(in_size, out_size, kernel_size=1),
        nn.BatchNorm1d(out_size)
    )

In [248]:
# Основной блок B_i
class QnetBlock(nn.Module):
    def __init__(self, in_size, out_size, kernel_size, stride=1,
                R=5):
        super().__init__()
        
        self.layers = nn.ModuleList(sepconv_bn(in_size, out_size, kernel_size, stride))
        for i in range(R - 1):
            self.layers.append(nn.ReLU())
            self.layers.append(sepconv_bn(out_size, out_size, kernel_size, stride))
        self.layers = nn.Sequential(*self.layers)
        
        self.residual = nn.ModuleList()
        self.residual.append(torch.nn.Conv1d(in_size, out_size, kernel_size=1))
        self.residual.append(torch.nn.BatchNorm1d(out_size))
        self.residual = nn.Sequential(*self.residual)
    
    def forward(self, x):
        return F.relu(self.residual(x) + self.layers(x))

In [249]:
class QuartzNet(nn.Module):
    def __init__(self, n_mels, num_classes):
        super().__init__()
                    
        self.c1 = sepconv_bn(n_mels, 256, kernel_size=33, stride=2)
                  
        self.blocks = nn.Sequential(
                #         in   out  k   s  R
                QnetBlock(256, 256, 33, 1, R=5),
                QnetBlock(256, 256, 39, 1, R=5),
                QnetBlock(256, 512, 51, 1, R=5),
                QnetBlock(512, 512, 63, 1, R=5),
                QnetBlock(512, 512, 75, 1, R=5)
        )
        
        self.c2 = sepconv_bn(512, 512, kernel_size=87, dilation=2, padding=86)
        
        self.c3 = conv_bn_act(512, 1024, kernel_size=1)
        
        self.c4 = conv_bn_act(1024, num_classes, kernel_size=1)
        
        self.init_weights()


    def init_weights(self):
        pass


    def forward(self, x):
        c1 = F.relu(self.c1(x))
        blocks = self.blocks(c1)
        c2 = F.relu(self.c2(blocks))
        c3 = self.c3(c2)
        return self.c4(c3)

#### обратим внимание, что c1 и c2 тоже separable

### Обучение 

In [242]:
melspec = torchaudio.transforms.MelSpectrogram(
    sample_rate=16000,           ### 22050, 48000
    n_fft=1024,
    hop_length=256,
    n_mels=N_MELS                ### 64,    80
).to(device)


# with augmentations
melspec_transforms = nn.Sequential(
    torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_fft=1024, hop_length=256,  n_mels=N_MELS),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=15),
    torchaudio.transforms.TimeMasking(time_mask_param=35),
).to(device)

In [251]:
def train_epoch(model, optimizer, dataloader, CTCLoss, device, melspec_transforms):
    model.train()
    
    losses = []
    
    for i, (wavs, wavs_len, answ, answ_len) in tqdm(enumerate(dataloader)):
        wavs, answ = wavs.to(device), answ.to(device)
        
        trans_wavs = torch.log(melspec_transforms(wavs) + 1e-9)
        
        optimizer.zero_grad()
            
        output = model(trans_wavs)
        output = F.log_softmax(output, dim=1)
        output = output.transpose(0, 1).transpose(0, 2)        
        
        loss = CTCLoss(output, answ, wavs_len, answ_len)
        loss.backward()        
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 15)
        optimizer.step()
        losses.append(loss.item())
        if i % 100 == 0:
            wandb.log({'mean_train_loss':loss})
            preds, targets = decoder_func(output, answ, answ_len, del_repeated=False)
            wandb.log({"CER_train": cer(targets[0], preds[0])})
            wandb.log({"WER_train": wer(targets[0], preds[0])})
        
    return np.mean(losses)

In [252]:
def train(model, opt, train_dl, scheduler, CTCLoss, device, n_epochs, val_dl=None):
    for epoch in range(n_epochs):
        print("Epoch {} of {}".format(epoch, n_epochs), 'LR', scheduler.get_last_lr())
        
        mean_loss = train_epoch(model, opt, train_dl, CTCLoss, device)
        print('MEAN EPOCH LOSS IS', mean_loss)
        
        scheduler.step()
        
        if (val_dl != None):
            test(model, opt, val_dl, CTCLoss, device)
            
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'scheduler_state_dict': scheduler.state_dict()
        }, 'epoch_5_and_'+str(epoch))

In [253]:
def decoder_func(output, answ, answ_lens, blank_label=0, del_repeated=True):    
    decoded_preds, decoded_targs = [], []
    
    text_transform = TextTransform()

    batch_freqs = torch.argmax(output, dim=2).transpose(0, 1)
        
    for i, freqs in enumerate(batch_freqs):
        preds = []
        
        decoded_targs.append(
            text_transform.int_to_text(answ[i][:answ_lens[i]].tolist())
        )
        
        for j, num in enumerate(freqs):
            if num != blank_label:
                if del_repeated and j != 0 and num == freqs[j-1]:
                    continue
                preds.append(num.item())
        decoded_preds.append(text_transform.int_to_text(preds))

    return decoded_preds, decoded_targs    

In [254]:
def cer(target, pred):
    cer_res = asrtoolkit.cer(''.join(target), ''.join(pred))
    return cer_res

def wer(target, pred):
    wer_res = asrtoolkit.wer(''.join(target), ''.join(pred))
    return wer_res

In [282]:
def test(model, optimizer, dataloader, CTCLoss, device, melspec, bs_width=None):
    model.eval()
    
    cers, wers, cers_bs, wers_bs = [], [], [], []
    losses = []
    
    with torch.no_grad():
        for i, (wavs, wavs_len, answ, answ_len) in enumerate(dataloader):
            wavs, answ = wavs.to(device), answ.to(device)

            trans_wavs = torch.log(melspec(wavs) + 1e-9)

            output = model(trans_wavs)
            if bs_width != None:
                output_bs = F.softmax(output, dim=1).transpose(0, 1).transpose(0, 2)
                preds_bs, targets_bs = beam_search_decoding(output_bs, answ, answ_len, width=bs_width)
                
            output = F.log_softmax(output, dim=1)
            output = output.transpose(0, 1).transpose(0, 2)
            loss = CTCLoss(output, answ, wavs_len, answ_len)
            losses.append(loss.item())
            
            # argmax
            preds, targets = decoder_func(output, answ, answ_len, del_repeated=True)
            # beam search
            
            for i in range(len(preds)):
                if i == 0:
                    print('target: ', ''.join(targets[i]))
                    print('prediction: ', ''.join(preds[i]))
                    print('beam_search_preds: ', ''.join(preds_bs[i]))
                    
                cers.append(cer(targets[i], preds[i]))
                wers.append(wer(targets[i], preds[i]))
                cers_bs.append(cer(targets_bs[i], preds_bs[i]))
                wers_bs.append(wer(targets_bs[i], preds_bs[i]))
                
        avg_cer = np.mean(cers)
        avg_wer = np.mean(wers)
        if bs_width != None:
            avg_cer_bs = np.mean(cers_bs)
            avg_wer_bs = np.mean(wers_bs)

        
        print('CER', avg_cer)
        print('CER BS', avg_cer_bs)
        
        print('WER', avg_wer)
        print('WER BS', avg_wer_bs)
        #wandb.log({"CER_val": avg_cer})    
        #wandb.log({"WER_val": avg_wer})
        avg_loss= np.mean(losses)
        print('average test loss is', avg_loss)
        #wandb.log({'mean_VAL_loss':avg_loss})


In [256]:
wandb.login()
wandb.init()
train_table = wandb.Table(columns=["Predicted Text", "True Text"])

NameError: name 'wandb' is not defined

In [257]:
model = QuartzNet(n_mels=64, num_classes=28)
print(count_parameters(model))
model.to(device)
wandb.watch(model)

In [259]:
#model = QuartzNet(n_mels=64, num_classes=28)
#num_of_params = count_parameters(model)
#model.to(device)
#wandb.watch(model)
#opt = torch.optim.RMSprop(model.parameters(), weight_decay=0.0001)
#scheduler = StepLR(opt, step_size=1, gamma=0.9) 
#checkpoint = torch.load('epoch_5', map_location=torch.device('cpu'))
#model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [260]:
opt = torch_optimizer.NovoGrad(
                        model.parameters(),
                        lr=0.01,
                        betas=(0.8, 0.5),
                        weight_decay=0.001,
)
scheduler  = CosineAnnealingLR(opt, T_max=50, eta_min=0, last_epoch=-1) # ###### TMAX = MAX NUM OF EPOCHS

#opt = torch.optim.RMSprop(model.parameters(), weight_decay=0.0001)
#scheduler = StepLR(opt, step_size=1, gamma=0.9) 

In [261]:
CTCLoss = nn.CTCLoss(blank=0).to(device)

In [ ]:
#train(model, opt, train_loader, scheduler, CTCLoss, device,
#     n_epochs=NUM_EPOCHS, val_dl=val_loader)

0it [00:00, ?it/s]

Epoch 0 of 150 LR [0.01]


173it [10:03,  3.52s/it]

torch.Size([2, 180864])
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -4.0159e-05,
         -3.1214e-05,  3.6880e-06],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -4.0159e-05,
         -3.1214e-05,  3.6880e-06]])


231it [13:26,  3.48s/it]

...

target oconnor charity fund
prediction the corer hurdy fomehlt 
target the following years were not nearly as successful
prediction thefo ears were not narly as sucessful
target the schools mascot is the rising sun
prediction the scoos masicot is the rising so
target he was replaced by new lead curtis young
prediction he was supplaed by nuele crutes guns
target collins and major chuck yeager
prediction cos an majer trip eater
target he then went on to direct minor films
prediction he te worin to tarit monfils
target grusin is married to nan newton
prediction rruson is married to nan muvise
target text programs are easier to write smaller and run faster
prediction theprorensar isi to rraloter abepesiten
target a woman in a white shirt playing volleyball
prediction e bomen in e wa shirt paying voyble
target he was left in his cage
prediction he was te lefst inus cae
target these are available in both print and electronically
prediction the arrawasbu trit armetor
target he received his in

target it tastes strongly of iodine
prediction it a stoly ofaedise
target dibens lives and trains in boulder colorado
prediction dibins livves an traineds in bulder colleroa
target the contact was limited but it was at full speed
prediction the pontic was ebecatebeytaser fu spedy
target three people giving a presentation before a crowd
prediction fre pepeliing of pisicatiton e for
target they toured around europe and north america mostly as support
prediction the tordaroour of ben northe merita mos le i suppor
target a celebration in king arthurs court follows
prediction a sobasion into otis co follos
target the group subsequently disbanded
prediction thet te coks ubchipentexunded
target it is eaten with bread
prediction it is acin ri brat
target the underfur is paler in color
prediction the counderfer is paler in coler
target the modern term for this meaning is ultrasonic
prediction the moden tem for the se is orte soni
target the toes contain lamellae
prediction he tos conan themello

0it [00:00, ?it/s]

target timothy played herriot in the television series
prediction twoesepla erriat in tytobision seris
average test loss is 1.3849328603336517
Epoch 8 of 150 LR [0.009381533400219317]


50it [02:53,  3.44s/it]

In [291]:
def beam_search_decoding(output, answ, answ_lens, blank_label=0, width=8):
    decoded_preds, decoded_targs  = [], []
    
    text_transform = TextTransform()    
    
    for i, mat in enumerate(output.transpose(0, 1)):
        last = {}        
        P_b, P_t = 1, 1
        P_nb = 0
        # dict       [0:prob_blank, 1:prob_not_blank, 2:prob_total, 3:list of ]
        last[''] = [P_b, P_nb, P_t]
                        
        for t in range(mat.shape[0]):
            curr = {}
            
            # sorting
            cand = [(key, el) for (key, el) in last.items()] # листы pb pnb pt beam
            sorted_cand = sorted(cand, reverse=True, key=lambda x: x[1][2]) # P_Total
            best_beams = [key for (key, el) in sorted_cand][0:width] # лучших w штук bшк
           
            for beam in best_beams:
                P_nb = 0
                if t == 0:
                    beam = ''
                else:
                    if len(beam) > 0:
                        last_num = text_transform.text_to_int(beam[-1])
                        P_nb = last[beam][1] * mat[t, last_num]

                    P_b = last[beam][2] * mat[t, blank_label]

                    if beam not in curr:
                        curr[beam] = [P_b, P_nb, P_b+P_nb]
                    else:
                        curr[beam][0] += P_b
                        curr[beam][1] += P_nb
                        curr[beam][2] += P_b + P_nb                
                
                # 0 is blank
                for c in range(1, mat.shape[1]): 
                    new_beam = beam + ''.join(text_transform.int_to_text([c]))                    
                    
                    if len(beam) > 0 and last_num == c:
                        P_nb = mat[t, c] * last[beam][0]
                    else:
                        P_nb = mat[t, c] * last[beam][2]
                    
                    if new_beam not in curr:
                        curr[new_beam] = [0, P_nb, P_nb]
                    else:
                        curr[new_beam][1] += P_nb 
                        curr[new_beam][2] += P_nb                
            last = curr
        
        cand = [(key, el) for (key, el) in last.items()]
        sorted_cand = sorted(cand, reverse=True, key=lambda x: x[1][2])
        best_beam = [x[0] for x in sorted_cand][0]
        
        decoded_preds.append(best_beam)
        
        # i - номер бача
        decoded_targs.append(
                text_transform.int_to_text(answ[i][:answ_lens[i]].tolist())
        )
        
    return decoded_preds, decoded_targs                

In [292]:
test(model, opt, val_loader, CTCLoss, device, bs_width=8)

target:  he started violently
prediction:  hs ter fi
beam_search_preds:  this ter fise
target:  i think it is
prediction:  i then etis
beam_search_preds:  bi  theng etes
torch.Size([2, 198144])
tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 4.8324e-05, 7.6901e-05,
         3.2309e-05],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 4.8324e-05, 7.6901e-05,
         3.2309e-05]])
target:  a boy is sleeping in his dinner bowl
prediction:  abi seping in as tat 
beam_search_preds:  ta bi seping in as ta nrllltttt                                                         
target:  a woman in gray cuts cake
prediction:  a won in gray cat
beam_search_preds:  ta  won in gray cats
target:  is she as good as you say
prediction:  the se as boreses
beam_search_preds:  the se  as boreses
target:  flaming youth
prediction:  flaiin ou
beam_search_preds:  ffllain ou
target:  list of mirmo
prediction:  he list bof e ammo
beam_search_preds:  the list  of e ammol
target:  you have a couch
prediction

target:  they are also used for local level polling

prediction:  they are also used for loka level polling